In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.10f}'.format

y'' -4 y' = x^6+1 ;


y(0) = 0 ; 


y'(0) = 3.56


{y''[x] - 4 y'[x] == x^6 + 1, y[0] == 0, y'[0] == 3.56}


In [25]:
# ВАРИАНТ 13
p = -4
q = 0 

c1 = (3.56+301/1024)/4

def analytic_answer(x):
    return -1/28 * x**7 - \
            1/16 * x**6 - \
            3/32 * x**5 - \
            15/128 * x**4 - \
            15/128 *  x**3 - \
            45/512* x**2 - \
            301/1024 *x + \
            c1* np.e**(4*x) + \
            (-c1)
def f(x):
    return x**6+1

In [3]:
# ВАРИАНТ 1
p=-2
q=1
def analytic_answer(x):
    return 1/2*np.e**x*(3*x*x + 2*x + 4)

def f(x):
    return 3*np.e**x

In [18]:
# ТЕСТ
def analytic_answer(x):
    return np.e**x

def f(x):
    return np.e**x

p = -1
q = 1

In [26]:
n = 10
h = 1/n
x0 = 0
y0 = analytic_answer(x0)
xn = 1
yn = analytic_answer(xn)
y0, yn

(0.0, 50.832909546512695)

In [27]:
def create_diagonals():
    top = []
    mid = []
    low = []
    res = []
    
    mid.append(h*h*q-2)
    top.append(1+h/2*p)
    
    x1 = x0 + (xn-x0)*h*1
    f1 = analytic_answer(x1)
    res.append(h*h*f1 - y0*(1-h/2*p))
    
    for i in range(2, n-1):
        low.append(1-h/2*p)
        mid.append(h*h*q-2)
        top.append(1+h/2*p)
        
        xi = x0 + (xn-x0)*h*i
        fi = f(xi)
        res.append(h*h*fi)
    
    low.append(1-h/2*p)
    mid.append(h*h*q-2)
    
    xn_1 = x0 + (xn-x0)*h*(n-1)
    fn_1 = analytic_answer(xn_1)
    res.append(h*h*fn_1 - yn*(1+h/2*p))
    
    return [0] + low, mid, top + [0], res
    
def solve_stripe(low, mid, top, res):
    n = len(mid)
    low = np.array(low)
    mid = np.array(mid)
    top = np.array(top)
    res = np.array(res)
    
    alpha = np.zeros((n,))
    beta = np.zeros((n,))
    alpha[0] = -top[0] / mid[0]
    beta[0] = res[0] / mid[0]

    for i in range(1, n):
        alpha[i] = -top[i]/(low[i]*alpha[i-1] + mid[i])
        beta[i] = (res[i] - low[i]*beta[i-1])/(low[i]*alpha[i-1] + mid[i])

    x = np.zeros((n,))
    x[n-1] = beta[n - 1]

    for i in range(n-1,0,-1):
        x[i-1] = alpha[i-1]*x[i] + beta[i-1]

    return list(x)

def create_matrix(low, mid, top):
    mat = np.diag(mid)
    for i in range(1,len(mid)):
        mat[i-1, i] = top[i-1]
        mat[i, i-1] = low[i]
    return mat

# убедимся, что работает
solve_stripe(
    low=[0,1,1,1],
    mid=[4,4,4,4],
    top=[1,1,1, 0],
    res=[5,6,6,5]
)

[1.0, 1.0, 1.0, 1.0]

In [28]:
low, mid, top, res = create_diagonals()
m = create_matrix(low,mid,top)
print(m)
res_my = np.array(
    [y0] + list(solve_stripe(low, mid, top, res)) + [yn]
)
res_my

[[-2.   0.8  0.   0.   0.   0.   0.   0.   0. ]
 [ 1.2 -2.   0.8  0.   0.   0.   0.   0.   0. ]
 [ 0.   1.2 -2.   0.8  0.   0.   0.   0.   0. ]
 [ 0.   0.   1.2 -2.   0.8  0.   0.   0.   0. ]
 [ 0.   0.   0.   1.2 -2.   0.8  0.   0.   0. ]
 [ 0.   0.   0.   0.   1.2 -2.   0.8  0.   0. ]
 [ 0.   0.   0.   0.   0.   1.2 -2.   0.8  0. ]
 [ 0.   0.   0.   0.   0.   0.   1.2 -2.   0.8]
 [ 0.   0.   0.   0.   0.   0.   0.   1.2 -2. ]]


array([ 0.        ,  0.42652447,  1.07185445,  2.05235023,  3.53560302,
        5.77303339,  9.14187426, 14.20821877, 21.82170615, 33.25771402,
       50.83290955])

In [29]:
res_true = np.array(
    [y0] + list(np.vectorize(analytic_answer)(np.linspace(0.1,1,9,False))) + [yn]
)
res_true

array([ 0.        ,  0.44346302,  1.11732778,  2.13491272,  3.66527756,
        5.96066496,  9.39747982, 14.537495  , 22.21927103, 33.69469807,
       50.83290955])

In [30]:
def shooting(Oh):
    y_0 = np.empty(n+1)
    y_1 = np.empty(n+1)
    
    y_0[0] = y0
    y_0[1] = y0 + Oh
    y_1[0] = 0
    y_1[1] = Oh

    for i in range(1, n):
        y_0[i+1] = (f(x0+i*h)*h**2 + (2-q*h**2)*y_0[i] - (1-p*h/2)*y_0[i-1]) / (1 + p*h/2)
        y_1[i+1] = ((2-q*h**2)*y_1[i] - (1-p*h/2)*y_1[i-1]) / (1 + p*h/2)

    if abs(y_1[n]) < 0.001:
        return gun(Oh+1)
    else:
        c1 = (yn - y_0[n]) / y_1[n]
    return [y_0[i] + c1 * y_1[i] for i in range(n+1)]
    
res_sh = shooting(h)
res_sh

[0.0,
 0.42547509255389093,
 1.0761877438847274,
 2.064757520880982,
 3.560121298875363,
 5.815718165866936,
 9.211808778854293,
 14.319027898335328,
 21.99382719005688,
 33.52180292763921,
 50.832909546512695]

In [36]:
df = pd.DataFrame({
    'xi': np.linspace(0,1,11,True),
    'analytic_ans': res_true,
    'run_ans': res_my,
    'run_abs_diff': np.abs(res_my-res_true),
    'sh_ans': res_sh,
    'sh_abs_diff': np.abs(res_sh-res_true),
})
print('Погрешность:', max(np.abs(res_my-res_true)))
print('Погрешность:', max(np.abs(res_sh-res_true)))
df.xi = df.xi.apply(lambda x: str(x)[:3])
df

Погрешность: 0.4369840463780079
Погрешность: 0.22544384247525784


,xi,analytic_ans,run_ans,run_abs_diff,sh_ans,sh_abs_diff
0,0.0,0.0000000000,0.0000000000,0.0000000000,0.0000000000,0.0000000000
1,0.1,0.4434630247,0.4265244660,0.0169385587,0.4254750926,0.0179879321
2,0.2,1.1173277845,1.0718544528,0.0454733318,1.0761877439,0.0411400406
3,0.3,2.1349127177,2.0523502329,0.0825624848,2.0647575209,0.0701551969
4,0.4,3.6652775563,3.5356030157,0.1296745406,3.5601212989,0.1051562574
5,0.5,5.9606649643,5.7730333898,0.1876315746,5.8157181659,0.1449467985
6,0.6,9.3974798232,9.1418742634,0.2556055598,9.2118087789,0.1856710444
7,0.7,14.5374950004,14.2082187739,0.3292762265,14.3190278983,0.2184671021
8,0.8,22.2192710325,21.8217061522,0.3975648803,21.9938271901,0.2254438425
9,0.9,33.6946980660,33.2577140196,0.4369840464,33.5218029276,0.1728951383
